In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

# Download stock data for a specific stock (e.g., Apple)
stock_data = yf.download('AAPL', start='2020-01-01', end='2023-01-01')

# Create features
# 1. Simple Moving Average (SMA)
stock_data['SMA_20'] = stock_data['Close'].rolling(window=20).mean()

# 2. Exponential Moving Average (EMA)
stock_data['EMA_20'] = stock_data['Close'].ewm(span=20, adjust=False).mean()

# 3. Relative Strength Index (RSI)
delta = stock_data['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)

avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()

rs = avg_gain / avg_loss
stock_data['RSI'] = 100 - (100 / (1 + rs))

# 4. Target variable: whether the next day's close price is higher (1) or lower (0) than today's close
stock_data['Target'] = np.where(stock_data['Close'].shift(-1) > stock_data['Close'], 1, 0)

# Drop rows with missing values (due to rolling and shifting operations)
stock_data = stock_data.dropna()

# Features and labels
X = stock_data[['SMA_20', 'EMA_20', 'RSI']]
y = stock_data['Target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create logistic regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Plot the actual vs predicted price movement
plt.figure(figsize=(10, 6))
plt.plot(stock_data.index[-len(y_test):], y_test, label="Actual", alpha=0.7)
plt.plot(stock_data.index[-len(y_test):], y_pred, label="Predicted", alpha=0.7)
plt.title('Stock Price Movement Prediction')
plt.legend()
plt.show()
